# hw4. Оценка качества исправления ошибок


1. Посмотрим на Trimmomatic(тупо обрезает), запуще его так:

``$ TrimmomaticPE -phred33 data/4/ecoli_400K_err_1.fastq data/4/ecoli_400K_err_2.fastq -baseout /home/toharhymes/work/ngs_ib_spring2020/project_four_errors/data/4/trimm_ecoli_400K LEADING:20 TRAILING:20  SLIDINGWINDOW:10:20 MINLEN:20``

Далее будем использовать только `trimm_ecoli_400K_1P` и `trimm_ecoli_400K_2P` (так как они у обоих ок).

Проиндексируем референс:

``$ bwa index data/4/MG1655-K12.first400K.fasta``

Выровняем скорректированные риды, и риды с ошибками:<br>
``$ bwa mem data/4/MG1655-K12.first400K.fasta data/4/trimm_ecoli_400K_1P data/4/trimm_ecoli_400K_2P > data/4/alignment_correct.sam`` <br>
``$ samtools flagstat data/4/alignment_correct.sam 
2689072 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2687824 + 0 mapped (99.95% : N/A)
2689072 + 0 paired in sequencing
1344536 + 0 read1
1344536 + 0 read2
2685506 + 0 properly paired (99.87% : N/A)
2686782 + 0 with itself and mate mapped
1042 + 0 singletons (0.04% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)``



``$ bwa mem data/4/MG1655-K12.first400K.fasta data/4/ecoli_400K_err_1.fastq data/4/ecoli_400K_err_2.fastq > data/4/alignment_error.sam ``<br>
``$ samtools flagstat data/4/alignment_error.sam 
2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
``

In [5]:
import pysam

import numpy as np
import pandas as pd

In [27]:
#возвращает словарь: индекс в референсе - значение(буква) в последовательности для рида
def make_index(read):
    dict_al = {}
    dict_ref = {}
    for read_i, ref_i in read.get_aligned_pairs(matches_only=False):
        dict_al.update({ref_i: None if read_i is None else read.query_sequene[read_i]})

def dunno(correct_sam_file, raw_sam_file):
    cor_sam = pysam.AlignmentFile(correct_sam_file, "r")
    raw_sam = pysam.AlignmentFile(raw_sam_file, "r")
    flag = True
    answer = np.zeros((2,3), dtype=int)
    try:
        while flag:
            cor_read = next(cor_sam)
            raw_read = next(raw_sam)
            if cor_read.is_unmapped or raw_read.is_unmapped:
                continue
            while cor_read.query_name != raw_read.query_name:
                for pair in raw_read.get_aligned_pairs(matches_only=False):
                    if pair[0] != None and pair[1] != None:
                        # Incorrectly removed base - убрал, а не должен был - сырой смапился 
                        answer[1][2] += 1
                    else:
                        # Correctly remove base - убрал правильно, так как не мапился, или мапился неверно
                        answer[0][2] += 1                    
                raw_read = next(raw_sam)
            cor_index = make_index(cor_read)
            raw_index = make_index(raw_read)
            ref_index
    except StopIteration:
        return answer
    

In [28]:
print(dunno('data/4/4/alignment_correct.sam', 'data/4/4/alignment_error.sam'))

[[     0      0  22378]
 [     0      0 159726]]


In [22]:
ans = np.zeros((2,3), dtype=int)
ans[1][1] += 3
ans[1][2] += 5
ans

array([[0, 0, 0],
       [0, 3, 5]])